In [1]:
import os

In [2]:
os.chdir('../')

In [3]:
from dataclasses import dataclass
from pathlib import  Path
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [4]:
from bpReviewClassifier.constants import *
from bpReviewClassifier.utils.common import read_yaml,create_directories

In [5]:
class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    def get_data_ingestion(self)->DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config


In [6]:
import os
import gzip
import shutil
import urllib.request as request
import zipfile
from bpReviewClassifier.logging import logger
from bpReviewClassifier.utils.common import get_size

In [7]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config=config
    def download_data(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers=request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded! with following info: \n{headers}")
        else:
            logger.info(f'file already exist of size {get_size(Path(self.config.local_data_file))}')
    def extract_zip_file(self):
        unzip_dir=self.config.unzip_dir
        os.makedirs(unzip_dir,exist_ok=True)
        file_path = os.path.join(unzip_dir, 'unzip_data')  
        with gzip.open(self.config.local_data_file, 'rb') as zip_ref:
             with open(file_path, 'wb') as decompressed_file:
                    shutil.copyfileobj(zip_ref, decompressed_file)
                    logger.info(f"GZ file extracted to {unzip_dir}")


        

In [8]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

[2025-01-18 16:30:42,114 : INFO : common : yaml file config\config.yaml loaded successfully]
[2025-01-18 16:30:42,119 : INFO : common : yaml file params.yaml loaded successfully]
[2025-01-18 16:30:42,122 : INFO : common : artifacts Created successfully.]
[2025-01-18 16:30:42,125 : INFO : common : artifacts/data_ingestion Created successfully.]
[2025-01-18 16:31:44,434 : INFO : 2450583016 : artifacts/data_ingestion/data.zip downloaded! with following info: 
Connection: close
Content-Length: 95357493
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/octet-stream
ETag: "f56ebc112b076e5b6ef551a2f9f65d715bb04cd250fa076c9ab57ed34aeebfe3"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 0DE6:16E4EB:4C0139:52513B:678B901F
Accept-Ranges: bytes
Date: Sat, 18 Jan 2025 11:30:44 GMT
Via: 1.1 varnish
X-Served-By: cach

PermissionError: [Errno 13] Permission denied: 'artifacts/data_ingestion/output'